In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import fetch_openml 
mnist = fetch_openml('mnist_784', as_frame=False)
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import time

index=np.random.randint(70000, size=1000)
data=mnist.data[index]
target=mnist.target[index]

#data = mnist.data
#target = mnist.target
pourcentage_de_données = 0.7
xtrain,  xtest,  ytrain,  ytest  =  train_test_split(data,  target, train_size = pourcentage_de_données)

noyaux=['sigmoid', 'poly', 'rbf', 'precomputed', 'linear']
c =['blue','red', 'green', 'pink', 'yellow', 'black', 'orange', 'purple', 'grey']

for n in noyaux:
    clf = SVC( kernel=n).fit(xtrain, ytrain)
    error=score(clf.predict(data),mnist.target)
    pr=precision_score(mnnist.target, clf.predict(data))
    rec=recall_score(mnist.target, clf.predict(data))
    plt.scatter(n, error, c='blue')
    plt.scatter(n, pr, c='green')
    plt.scatter(n, recall_score, c='purple')

plt.title('metrics depending on kernels')
plt.xlabel('kernel')
plt.ylabel('scores')
plt.show()

for n in noyaux:
    start=time.time()
    clf = SVC( kernel=n).fit(xtrain, ytrain)
    end=time.time()
    temps=end-start
    plt.scatter(n, temps)

plt.title('time depending on kernels')
plt.xlabel('kernel')
plt.ylabel('time')
plt.show()

for c in np.arange(0.2, 1, 0.2):
    clf = SVC( kernel='linear', C=c).fit(xtrain, ytrain)
    error=score(clf.predict(data),mnist.target)
    #nb de sets bien predits sur nb sets predits 
    #precision = Vrais Positifs / (Vrais Positifs + Faux Positifs)
    pr=precision_score(mnnist.target, clf.predict(data))
    rec=recall_score(mnist.target, clf.predict(data))
    #recall = Vrais Positifs / (Vrais Positifs + faux Negatifs) ,(deno=cste)
    #error=score qu on retrouve dans la fonction de perte qu on optimise 
    plt.scatter(n, error, c='blue')
    plt.scatter(n, pr, c='green')
    plt.scatter(n, rec, c='purple')

plt.title('metrics depending on kernels')
plt.xlabel('tolerance')
plt.ylabel('precision score')
plt.show()

for c in np.arange(0.2, 1, 0.2):
    start=time.time()
    clf = SVC( kernel='linear', C=c).fit(xtrain, ytrain)
    end=time.time()
    temps=end-start
    plt.scatter(n, temps)

plt.title('time depending on kernels')
plt.xlabel('tolerance')
plt.ylabel('time')
plt.show()


#cm = confusion_matrix(Y_test, Y_pred) 
